In [ ]:
#Image selection code
#Need to select image which of good quality and of the right area
#For each satellite overpass time check hour either side as sometimes the time is not exact at 30 mins past

#get a list of LST filenames

#open dataset and check the lat and long to see if the target coordinates are in the image

#create a list of the images with target coordinates
#then we need to check the quality of the city area
#select out the required area (city area)
#check the quality flags

#select file to be used if acceptable - add to a new list of filenames

In [1]:
#loop for Terra

import os
import xarray as xr 
import numpy as np



#count the number of LST files
#don't really need to do this
#Potentially remove this bit
#x = 0
#for filename in os.listdir(directory):
#    if filename.startswith("GT_SSD-L2-MOGSV_LST"):    #files containing LSTs
#        x += 1
#        #print(os.path.join(directory, filename))       
#    else:
#        continue
        
#add the filenames wanted to an array

directory = '/neodc/eustace/data/satellite_skin_temperature/UOL/land/MODIS_Terra/L2/GT_MOG_2P/v2.1/2007/07/15/'
LST_files = list()

for filename in os.listdir(directory):
    if filename.startswith("GT_SSD-L2-MOGSV_LST"):    #files containing LSTs
        pathname = os.path.join(directory, filename)
        LST_files.append(pathname)
    else:
        continue
        
city_lat = 51.508530
city_lon = -0.076132
city_files = list()

#get a list of files which contain the required area

for i in range(len(LST_files)):
    LST_filename = LST_files[i]
    ds = xr.open_dataset(LST_filename)
    if city_lat < ds.northernmost_latitude and city_lat > ds.southernmost_latitude and city_lon > ds.westernmost_longitude and city_lon < ds.easternmost_longitude:
        city_files.append(LST_filename)

#open up the city files and subset them. go half degree either side of the city centre (~55km either side)
#subsetting
max_lat = city_lat + 0.5
min_lat = city_lat - 0.5
max_lon = city_lon + 0.5
min_lon = city_lon - 0.5 


#defining the functions

#function to check the status of a byte on the QC control
def check_bits(x,n):
    if (x & (1<<n)): 
  ## n-th bit is 1 
        flag = 1
    else:
  ## n-th bit is 0
        flag = 0
    return flag

#function to check bytes 1 and 2 and return 0 if they are both 0
def quality_control(x):
    if ((check_bits(x,1) == 0) and (check_bits(x,2) == 0)):
  ##if bit 1 and bit 2 are 0 then 0 
        flag = 0
    else:
  ##otherwise flag is 1
        flag = 1
    return flag       

#check the QC flags
#create a new list for the files where more than 70% of the pixels are usable

quality_images = list()

for i in range(len(city_files)):
    city_filename = city_files[i]
    #load up file
    ds = xr.open_dataset(city_filename)
    #take a subset
    ds2 = ds.where((min_lon < ds.lon) & (ds.lon < max_lon)
               & (min_lat < ds.lat) & (ds.lat < max_lat), drop=True)
    #change to dataframe
    df2 = ds2.to_dataframe()
    #Take the QC flags out the dataframe to apply the function to 
    QC = df2.QC

    #if the flags are NaN set them to 111 (binary) = 7 (decimal) as this will be flagged as poor quality and therefore not used
    QC_no_nan =  [7 if np.isnan(x) == True else x for x in QC]

    #Apply to function to QC
    QC2 = np.array([quality_control(int(i)) for i in QC_no_nan])

    #And add this QC2 flag onto the dataset
    df2['QC_2'] = QC2

    #calculate fraction not useable 
    if len(df2.index) == 0:
        fractunusable = 1 
    else:
        fract_unusable = sum(df2['QC_2'])/ len(df2.index)

    #if 30% or more are not useable, the image should not be used
    #if the fraction unusable is less then 0.3 then add to a new list 

    if fract_unusable < 0.3:
        quality_images.append(city_filename)


In [37]:
quality_images

[]